### Prepping Data Challenge: Restaurant Menu & Orders (week 15)
This week we want to analyse the orders that customers have made over a period of time in our restaurant Serendipia. In order to identify how much money we earn each day of the week and also to discover who our top customer is. We are going to be using calculations, pivots and aggregations so lots of the fundamental techniques that are used within data prep! 

#### Requirement:
 - Input the data

 - Modify the structure of the Menu table so we can have one column for the Type (pizza, pasta, house plate), the name of the plate, ID, and Price 

 - Modify the structure of the Orders table to have each item ID in a different row 

 - Join both tables 

 - On Monday's we offer a 50% discount on all items. Recalculate the prices to reflect this

 - For Output 1, we want to calculate the total money for each day of the week 

 - For Output 2, we want to reward the customer who has made the most orders for their loyalty. Work out which customer has ordered the most single items. 

### Input the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
with pd.ExcelFile('WK15-Input.xlsx') as xlsx:
    menu = pd.read_excel(xlsx, 'MENU')
    orders = pd.read_excel(xlsx, 'Order', parse_dates=['Order Date'])

In [3]:
#menu.head()
#orders.head()

### Modify the structure of the Menu table so we can have one column for the Type (pizza, pasta, house plate), the name of the plate, ID, and Price 

In [4]:
menu = menu.reset_index().melt(id_vars=['index'])

In [5]:
menu[['Type', 'field']] = menu['variable'].str.extract('(.*?)\s?(ID|Price|$).*', expand=True)
menu['field'] = np.where(menu['field'].str.strip() == '', 'Item', menu['field'])
menu.dropna(subset=['value'], inplace=True)

In [6]:
# pivot fieldnames into columns
menu_f = menu.pivot_table(values='value', index=['index', 'Type'], columns=['field'],
                          aggfunc='first')\
             .reset_index()\
             .drop(columns='index')

In [7]:
# put each item ID from orders into a separate row
orders['Order'] = orders['Order'].astype(str).str.split('-')
orders = orders.explode('Order')
orders['Order'] = orders['Order'].astype(int)

###   Join both tables 

In [8]:
orders_menu = pd.merge(orders, menu_f, left_on='Order', right_on='ID', how='inner')

### On Monday's we offer a 50% discount on all items. Recalculate the prices to reflect this

In [9]:
orders_menu['Weekday'] = orders_menu['Order Date'].dt.day_name()
orders_menu['Price'] = np.where(orders_menu['Weekday'] == 'Monday', orders_menu['Price'] * 0.5,
                          orders_menu['Price'])

### For Output 1, we want to calculate the total money for each day of the week 

### For Output 2, we want to reward the customer who has made the most orders for their loyalty. Work out which customer has ordered the most single items. 

In [11]:
#output 1
output1 = orders_menu.groupby('Weekday')['Price'].sum().reset_index()

In [10]:
#output 2
output2 = orders_menu.groupby('Customer Name')['Order'].nunique().reset_index()\
               .sort_values(by='Order', ascending=False).iloc[0:1, :]
output2.rename(columns={'Order':'Count Items'}, inplace=True)

In [12]:
output1.to_csv('WK15-output1.csv', index = False)
output2.to_csv('Wk15-output2.csv', index = False)